<a href="https://colab.research.google.com/github/CosmoStat/Tutorials/blob/tensorflow-tutorial/TensorFlowFirstSteps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 Francois Lanusse, Zaccharie Ramzi

Licensed under the Apache License, Version 2.0 (the "License");

# First steps with TensorFlow

Authors: [@EiffL](https://github.com/EiffL) (Francois Lanusse), [@zaccharieramzi](https://github.com/zaccharieramzi) (Zaccharie Ramzi)

## Overview

In this short notebook, we will review the basics of TensorFlow, how to write a computational graph, compute gradients by automatic differentiation, and perform optimization. This introduction is intended for first time TensorFlow users, and can be safely disregarded by experienced users.  
Note: we only consider TensorFlow 2.x here.

### Learning goals

In this notebook, we will learn:
  - Core principles of TensorFlow
  - How to write a computational graph
  - How to compute gradients
  - How to perform optimization

## Importing TensorFlow  

- what is a tensor,
- how is it different than numpy
- how to go back and forth between the two

In [0]:
import tensorflow as tf
import numpy as np

#Checking for GPU access
if tf.test.gpu_device_name() != '/device:GPU:0':
  print('WARNING: GPU device not found.')
else:
  print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

SUCCESS: Found GPU: /device:GPU:0


If the GPU fails to load, activate the GPU hardware acceleration in Runtime > Change Runtime Type